# Compare Snow Depths with Interferogram

**Goal**: Compare the insar phase to snowdepths

**Approach**: 
1. Define an area to study the relationship 
2. Grab the snow depths from the magnaprobe in the area
3. Grab the real and imaginary pixels nearest the magna probe locations 
4. Plot it


### Step 1:  Define an Area to Compare Depths to Interferogram

In [1]:
from snowxsql.data import PointData, ImageData, SiteData 
from snowxsql.db import get_db
from snowxsql.conversions import points_to_geopandas
from snowxsql.functions import ST_PixelAsPoint
import geoalchemy2.functions as gfunc
from geoalchemy2.types import Raster
from geoalchemy2.shape import to_shape

from sqlalchemy.sql import func
import geopandas as gpd
import numpy as np 

# PIT Site Identifier
site_id = '5S31'

# Distance around the pit to collect data in meters
buffer_dist = 50

# Connect to the database we made.
db_name = 'snowex'
engine, session = get_db(db_name)

# Grab our pit location by provided site id from the site details table
q = session.query(SiteData).filter(SiteData.site_id == site_id)
sites = q.all()
print(session.query(gfunc.ST_AsText(sites[0].geom)).all())
# There can be different dates at a single site, so we only grab one to retrieve the geometry object
point = session.query(sites[0].geom.ST_AsText()).limit(1).all()
print(point)

# Create a polygon buffered by our distance centered on the pit
q = session.query(gfunc.ST_Buffer(point[0][0], buffer_dist))
buffered_pit = q.all()[0][0]
print(to_shape(buffered_pit,srid=26912))

[('POINT(745458 4322762)',)]
[('POINT(745458 4322762)',)]


TypeError: to_shape() got an unexpected keyword argument 'srid'

### Step 2: Grab all the Snow Depths in the Area

In [ ]:
# Grab all the point data in the buffer
points = session.query(PointData).filter(gfunc.ST_Within(PointData.geom.ST_AsText(), buffered_pit.ST_AsText())).all()

# Convert the records received to geopandas
df = points_to_geopandas(points)

### Step 3: Grab near polygons of pixels in the area

In [ ]:
# Loop over all the points
session.rollback()
q = session.query(func.ST_PixelAsPolygons(func.ST_Union(ImageData.raster, type_=Raster)))
q = q.filter(ImageData.type == 'insar interferogram imaginary')
q = q.filter(gfunc.ST_Intersects(ImageData.raster, buffered_pit))
pixels = q.all()
print(pixels)

#pixel = session.query(ST_PixelAsPoint(ImageData.raster,1 ,1)).filter(ImageData.type.contains('interferogram')).limit(1).all()
#print(pixel)